In [23]:
import pandas as pd

path = r"C:\Users\Eduardo\Downloads\nbbo_20140131.sas7bdat"

df = pd.read_sas(path, format="sas7bdat", encoding="latin1")

df.head(-10)


,SYMBOL,DATE,TIME,BB,BBSize,BBASize,BBEXLIST,BO,BOSize,BOASize,BOEXLIST,NUMEX
0,A,2014-01-31,14401.0,NaN,NaN,NaN,NaN,63.12,1.0,1.0,P,1.0
1,A,2014-01-31,22284.0,NaN,NaN,NaN,NaN,59.22,2.0,2.0,P,1.0
2,A,2014-01-31,22288.0,NaN,NaN,NaN,NaN,63.12,1.0,1.0,P,1.0
3,A,2014-01-31,22306.0,NaN,NaN,NaN,NaN,59.28,1.0,1.0,P,1.0
4,A,2014-01-31,22311.0,NaN,NaN,NaN,NaN,63.12,1.0,1.0,P,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
49374544,ZXZZT,2014-01-31,61188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
49374545,ZXZZT,2014-01-31,61189.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
49374546,ZXZZT,2014-01-31,61190.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
49374547,ZXZZT,2014-01-31,61191.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [24]:

# 2) Asegurarnos de que DATE es un datetime a medianoche
#    pandas.read_sas suele devolverlo como datetime64[ns] ya.
#    Si no fuera así (si es string YYYY-MM-DD), entonces:
df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")

# 3) TIME lo convertimos a número de segundos desde la medianoche.
#    Inspecciona df["TIME"].dtype y unos valores con:
print(df["TIME"].head(), df["TIME"].dtype)

#    Asumamos que TIME está en segundos (p.ej. 14 401.0  →  04:00:01)
df["time_secs"] = df["TIME"].astype(float).round().astype(int)

# 4) Creamos la columna datetime sumando el timedelta
df["datetime"] = df["DATE"] + pd.to_timedelta(df["time_secs"], unit="s")

df = df.set_index("datetime")

# 3) Calcular mid_price
df["mid"] = (df["BB"] + df["BO"]) / 2

#4) Filtrar por símbolo “A” (o el que tú quieras)
df = df[df["SYMBOL"] == "A"].sort_index()

0    14401.0
1    22284.0
2    22288.0
3    22306.0
4    22311.0
Name: TIME, dtype: float64 float64


In [26]:
# Ejemplo: resample a cada 60 segundos tomando el último mid
X = "60S"  # 60 segundos; podrías usar "5S" para 5 segundos, "1T" para 1 minuto…
df_rsp = df.resample(X).agg({
    "mid": "last",
    "BBSize": "sum",
    "BOSize": "sum",
    "BB": "mean",
    "BO": "mean"
})
df_rsp = df_rsp.dropna(subset=["mid"])
# Ahora df_rsp tiene ~ n_rows = total_duration_seconds / 60
print(df_rsp.shape)

(469, 5)


C:\Users\Eduardo\AppData\Local\Temp\ipykernel_19300\4114848596.py:3: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df_rsp = df.resample(X).agg({


In [27]:
# Feature engineering
df_rsp["spread"]    = df_rsp["BO"] - df_rsp["BB"]
df_rsp["imbalance"] = (df_rsp["BBSize"] - df_rsp["BOSize"]) / (df_rsp["BBSize"] + df_rsp["BOSize"])

# Target: mid-price un paso X en el futuro
df_rsp["mid_future"] = df_rsp["mid"].shift(-1)  # un paso hacia adelante en el resample

# Quitar la última fila (target NaN)
df_ml = df_rsp.dropna(subset=["mid_future"])

In [28]:
from sklearn.model_selection import train_test_split

features = ["mid", "spread", "imbalance", "BBSize", "BOSize"]
X = df_ml[features]
y = df_ml["mid_future"]

# Con shuffle=False para respetar orden temporal
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

In [29]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

pipe = make_pipeline(
    StandardScaler(),
    RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=-1)
)

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

# mean_squared_error sin argumento `squared`, luego aplicamos sqrt
mse  = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("RMSE:", rmse)
print("R²:", r2_score(y_test, y_pred))


RMSE: 0.2109756635779629
R²: 0.7116850982159506


In [30]:
last_row = df_rsp.iloc[-1]  # la última ventana de 60 s
feat = last_row[["mid", "spread", "imbalance", "BBSize", "BOSize"]].values.reshape(1,-1)

next_mid_pred = pipe.predict(feat)[0]
print("Predicción del mid para el próximo minuto:", next_mid_pred)


Predicción del mid para el próximo minuto: 58.96360000000004


c:\Users\Eduardo\Desktop\facultad\ciencia-de-datos\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [32]:
df_eval = X_test.copy()

# mid_actual está en la columna "mid" de X_test
# mid-price en el instante t (por ejemplo a las 16:44:00).
df_eval["mid_actual"] = df_eval["mid"]

# mid_true es el valor real de y_test
# mid-price que realmente ocurrió en t+1 (por ejemplo a las 16:45:00).
df_eval["mid_true"] = y_test.values

# mid_pred es tu predicción
# mid-price que tu modelo predijo para ese t+1.
df_eval["mid_pred"] = y_pred

# 2) Ahora mostramos las últimas 30 filas
df_eval[["mid_actual","mid_true","mid_pred"]].tail(30)

,mid_actual,mid_true,mid_pred
datetime,,,
2014-01-31 16:24:00,58.410,58.410,58.366350
2014-01-31 16:25:00,58.410,58.360,58.357400
2014-01-31 16:26:00,58.360,58.400,58.367700
2014-01-31 16:27:00,58.400,58.370,58.351000
2014-01-31 16:28:00,58.370,58.360,58.365450
2014-01-31 16:29:00,58.360,58.390,58.370800
2014-01-31 16:30:00,58.390,58.370,58.354650
2014-01-31 16:31:00,58.370,58.390,58.357697
2014-01-31 16:32:00,58.390,58.380,58.359060


In [33]:
from sklearn.metrics import mean_squared_error
import numpy as np

residuals = y_test - y_pred

mse  = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae  = np.mean(np.abs(residuals))
mape = np.mean(np.abs(residuals / y_test)) * 100

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"R²: {r2_score(y_test, y_pred):.4f}")

MSE: 0.0445
RMSE: 0.2110
MAE: 0.1309
MAPE: 0.22%
R²: 0.7117
